In [2]:
import tensorflow as tf
# import tensorflow_hub as hub
# from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

print("TF version:", tf.__version__)
# print("Hub version:", hub.__version__)


2024-05-20 12:43:44.148340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 12:43:44.316049: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 12:43:44.320848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-20 12:43:44.320863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart d

TF version: 2.10.0


In [4]:

# Download the model 
!wget -nc -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite

# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}


In [7]:
model_input_size = 192
sides = ["left" , "right"]
body_parts = ["shoulder", "hip", "ankle", "knee"]


def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()    
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores
    

def read_image(image_path):
    # Show the image

    import matplotlib.image as mpimg
    image = mpimg.imread(image_path)
    
    return image    

def convert_to_movenet_format(image):
    expanded_image = tf.expand_dims(image, axis=0)    
    resized_image = tf.image.resize_with_pad(expanded_image, model_input_size, model_input_size)
    return resized_image


def reoganize_output(movenet_output):
    points = dict()
    
    for side in sides:
        points[side] = dict()
        for body_part in body_parts:
            points[side][body_part] = movenet_output[0][0][KEYPOINT_DICT[f"{side}_{body_part}"]]

    return points


def annotate_and_show(image, points):

    height, width, _ = image.shape
    
    plt.imshow(image /255.0)
    
    for side in sides:
        for body_part in body_parts:            
            absolute_x = points[side][body_part][1] * width
            absolute_y = points[side][body_part][0] * height
            # print(f"{side}_{body_part}, x = {absolute_x} , y = {absolute_y}")
            if side == "left":
                plt.plot(absolute_x, absolute_y, 'bo')
            else:
                plt.plot(absolute_x, absolute_y, 'ro')
    
    plt.show()


    